In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("./datasets/vehicles_2012-13/vehicles_2012.csv")
df1 = pd.read_csv("./datasets/vehicles_2012-13/vehicles_2013.csv")
result = df.append(df1)


/tmp/ipykernel_23181/2731977298.py:1: DtypeWarning: Columns (5,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./datasets/vehicles_2012-13/vehicles_2012.csv")
/tmp/ipykernel_23181/2731977298.py:2: DtypeWarning: Columns (5,9,13,14,15,19,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("./datasets/vehicles_2012-13/vehicles_2013.csv")
/tmp/ipykernel_23181/2731977298.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = df.append(df1)


In [5]:
result

,Unnamed: 0,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
0,0,10385780,100201,2012-09-07,9:03,1,NY,PASSENGER VEHICLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN
1,9,11804847,2975897,2012-11-26,18:12,2,NY,PASSENGER VEHICLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Driver Inattention/Distraction,NaN
2,15,11912713,176016,2012-08-11,19:36,2,NY,BICYCLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN
3,30,8393362,1113,2012-10-26,16:20,2,NY,BICYCLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN
4,40,10957637,126113,2012-09-04,7:00,2,NY,SPORT UTILITY / STATION WAGON,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lost Consciousness,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404680,3079374,18800351,4028370,2013-11-13,23:10,b44e5be6-05f3-4d79-824d-c3994bc901ba,NaN,Bike,NaN,NaN,...,Going Straight Ahead,Right Front Quarter Panel,Right Front Quarter Panel,Right Front Quarter Panel,Right Front Quarter Panel,No Damage,N,NaN,Driver Inattention/Distraction,Driver Inattention/Distraction
404681,3092119,19074461,4199221,2013-09-03,18:25,422aa181-be94-49c8-99c6-8f2aa128afe4,NY,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,NaN,...,Stopped in Traffic,Right Front Bumper,Right Front Bumper,NaN,NaN,NaN,N,NaN,Unspecified,Unspecified
404682,3106374,19031143,4178777,2013-07-25,12:45,95cab9a6-519b-4eab-9b09-3b3cade8fcc6,PA,Sedan,CHEV -CAR/SUV,NaN,...,Going Straight Ahead,Right Front Bumper,Right Front Bumper,No Damage,No Damage,No Damage,N,NaN,Unspecified,Unspecified
404683,3124597,18789056,4031591,2013-11-23,20:40,31dafa15-41cc-45de-895d-af5589157f25,NY,Taxi,TOYT -CAR/SUV,NaN,...,Slowing or Stopping,Left Rear Bumper,Left Rear Bumper,NaN,NaN,NaN,N,NaN,Unspecified,Unspecified


In [6]:
result.isna().sum()

Unnamed: 0                          0
UNIQUE_ID                           0
COLLISION_ID                        0
CRASH_DATE                          0
CRASH_TIME                          0
VEHICLE_ID                          0
STATE_REGISTRATION               1384
VEHICLE_TYPE                      325
VEHICLE_MAKE                   603536
VEHICLE_MODEL                  603652
VEHICLE_YEAR                   603536
TRAVEL_DIRECTION               603522
VEHICLE_OCCUPANTS              603527
DRIVER_SEX                     603552
DRIVER_LICENSE_STATUS          603558
DRIVER_LICENSE_JURISDICTION    603556
PRE_CRASH                      603496
POINT_OF_IMPACT                603525
VEHICLE_DAMAGE                 603525
VEHICLE_DAMAGE_1               603569
VEHICLE_DAMAGE_2               603586
VEHICLE_DAMAGE_3               603598
PUBLIC_PROPERTY_DAMAGE         603517
PUBLIC_PROPERTY_DAMAGE_TYPE    603652
CONTRIBUTING_FACTOR_1            2924
CONTRIBUTING_FACTOR_2          603524
dtype: int64

In [7]:
result = result.dropna(subset=['VEHICLE_TYPE'])

In [8]:
result

,Unnamed: 0,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
0,0,10385780,100201,2012-09-07,9:03,1,NY,PASSENGER VEHICLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN
1,9,11804847,2975897,2012-11-26,18:12,2,NY,PASSENGER VEHICLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Driver Inattention/Distraction,NaN
2,15,11912713,176016,2012-08-11,19:36,2,NY,BICYCLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN
3,30,8393362,1113,2012-10-26,16:20,2,NY,BICYCLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN
4,40,10957637,126113,2012-09-04,7:00,2,NY,SPORT UTILITY / STATION WAGON,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lost Consciousness,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404680,3079374,18800351,4028370,2013-11-13,23:10,b44e5be6-05f3-4d79-824d-c3994bc901ba,NaN,Bike,NaN,NaN,...,Going Straight Ahead,Right Front Quarter Panel,Right Front Quarter Panel,Right Front Quarter Panel,Right Front Quarter Panel,No Damage,N,NaN,Driver Inattention/Distraction,Driver Inattention/Distraction
404681,3092119,19074461,4199221,2013-09-03,18:25,422aa181-be94-49c8-99c6-8f2aa128afe4,NY,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,NaN,...,Stopped in Traffic,Right Front Bumper,Right Front Bumper,NaN,NaN,NaN,N,NaN,Unspecified,Unspecified
404682,3106374,19031143,4178777,2013-07-25,12:45,95cab9a6-519b-4eab-9b09-3b3cade8fcc6,PA,Sedan,CHEV -CAR/SUV,NaN,...,Going Straight Ahead,Right Front Bumper,Right Front Bumper,No Damage,No Damage,No Damage,N,NaN,Unspecified,Unspecified
404683,3124597,18789056,4031591,2013-11-23,20:40,31dafa15-41cc-45de-895d-af5589157f25,NY,Taxi,TOYT -CAR/SUV,NaN,...,Slowing or Stopping,Left Rear Bumper,Left Rear Bumper,NaN,NaN,NaN,N,NaN,Unspecified,Unspecified


In [9]:
result_grped = result.groupby('VEHICLE_TYPE').count()
result_grped

,Unnamed: 0,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_YEAR,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
VEHICLE_TYPE,,,,,,,,,,,,,,,,,,,,,
4 dr sedan,6,6,6,6,6,6,6,6,0,6,...,6,6,6,4,3,2,6,0,6,6
AMBULANCE,1244,1244,1244,1244,1244,1244,1244,0,0,0,...,0,0,0,0,0,0,0,0,1228,0
BICYCLE,7997,7997,7997,7997,7997,7997,7262,0,0,0,...,1,0,0,0,0,0,0,0,7963,0
BUS,9816,9816,9816,9816,9816,9816,9815,0,0,0,...,0,0,0,0,0,0,0,0,9677,0
Bike,6,6,6,6,6,6,1,0,0,0,...,6,5,5,3,3,3,6,0,6,6
Box Truck,8,8,8,8,8,8,7,7,0,7,...,8,8,8,6,5,4,8,0,8,8
Convertible,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,0,1,1
E-Scooter,1,1,1,1,1,1,0,0,0,0,...,1,1,1,0,0,0,1,0,1,1
FIRE TRUCK,521,521,521,521,521,521,521,0,0,0,...,0,0,0,0,0,0,0,0,515,0


In [10]:
# grouping few of the categories can be a good option and then we can proceed with the inferences.
def month_based(value):
    if str(value)[5:7]=="01":
        return "January"
    elif str(value)[5:7]=="02":
        return "February"
    elif str(value)[5:7]=="03":
        return "March"
    elif str(value)[5:7]=="04":
        return "April"
    elif str(value)[5:7]=="05":
        return "May"
    elif str(value)[5:7]=="06":
        return "June"
    elif str(value)[5:7]=="07":
        return "July"
    elif str(value)[5:7]=="08":
        return "August"
    elif str(value)[5:7]=="09":
        return "September"
    elif str(value)[5:7]=="10":
        return "October"
    elif str(value)[5:7]=="11":
        return "November"
    elif str(value)[5:7]=="12":
        return "December"


In [11]:
result["ACCIDENT_MONTH"] = result["CRASH_DATE"].map(month_based)
result

/tmp/ipykernel_23181/3487122884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["ACCIDENT_MONTH"] = result["CRASH_DATE"].map(month_based)


,Unnamed: 0,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,...,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,ACCIDENT_MONTH
0,0,10385780,100201,2012-09-07,9:03,1,NY,PASSENGER VEHICLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN,September
1,9,11804847,2975897,2012-11-26,18:12,2,NY,PASSENGER VEHICLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Driver Inattention/Distraction,NaN,November
2,15,11912713,176016,2012-08-11,19:36,2,NY,BICYCLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN,August
3,30,8393362,1113,2012-10-26,16:20,2,NY,BICYCLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN,October
4,40,10957637,126113,2012-09-04,7:00,2,NY,SPORT UTILITY / STATION WAGON,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lost Consciousness,NaN,September
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404680,3079374,18800351,4028370,2013-11-13,23:10,b44e5be6-05f3-4d79-824d-c3994bc901ba,NaN,Bike,NaN,NaN,...,Right Front Quarter Panel,Right Front Quarter Panel,Right Front Quarter Panel,Right Front Quarter Panel,No Damage,N,NaN,Driver Inattention/Distraction,Driver Inattention/Distraction,November
404681,3092119,19074461,4199221,2013-09-03,18:25,422aa181-be94-49c8-99c6-8f2aa128afe4,NY,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,NaN,...,Right Front Bumper,Right Front Bumper,NaN,NaN,NaN,N,NaN,Unspecified,Unspecified,September
404682,3106374,19031143,4178777,2013-07-25,12:45,95cab9a6-519b-4eab-9b09-3b3cade8fcc6,PA,Sedan,CHEV -CAR/SUV,NaN,...,Right Front Bumper,Right Front Bumper,No Damage,No Damage,No Damage,N,NaN,Unspecified,Unspecified,July
404683,3124597,18789056,4031591,2013-11-23,20:40,31dafa15-41cc-45de-895d-af5589157f25,NY,Taxi,TOYT -CAR/SUV,NaN,...,Left Rear Bumper,Left Rear Bumper,NaN,NaN,NaN,N,NaN,Unspecified,Unspecified,November


In [12]:
result_grped = result.groupby('ACCIDENT_MONTH').count()
result_grped

,Unnamed: 0,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
ACCIDENT_MONTH,,,,,,,,,,,,,,,,,,,,,
April,32721,32721,32721,32721,32721,32721,32647,32721,3,1,...,3,3,3,0,0,0,3,0,32522,3
August,67593,67593,67593,67593,67593,67593,67427,67593,16,0,...,25,19,19,14,12,11,19,0,67326,19
December,67642,67642,67642,67642,67642,67642,67517,67642,20,0,...,25,21,21,16,14,12,21,0,67334,21
Febraury,28349,28349,28349,28349,28349,28349,28293,28349,6,0,...,8,6,6,4,4,3,6,0,28207,6
January,30876,30876,30876,30876,30876,30876,30801,30876,0,0,...,0,0,0,0,0,0,0,0,30767,0
July,69093,69093,69093,69093,69093,69093,68901,69093,18,0,...,24,20,20,13,11,8,20,0,68814,20
June,36272,36272,36272,36272,36272,36272,36153,36272,9,0,...,11,9,9,6,4,4,9,0,36027,9
March,32813,32813,32813,32813,32813,32813,32751,32813,2,0,...,7,2,2,1,1,1,3,0,32649,3
May,36862,36862,36862,36862,36862,36862,36777,36862,4,0,...,5,4,4,1,0,0,4,0,36561,4
